In [45]:
#Khai báo thư viện
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.functions import col, when, isnull
from pyspark.sql import functions as F
from pyspark.sql.functions import desc, asc
import matplotlib.pyplot as plt
import pandas as pd

In [46]:
#Đọc file csv được thu thập từ trang https://finance.yahoo.com/
spark = SparkSession.builder.getOrCreate()
data = spark.read.csv("AAPL.csv", header=True, inferSchema=True)
data = data.drop("Adj Close")

In [47]:
data.toPandas()

,Date,Open,High,Low,Close,Volume
0,1980-12-12,0.128348,0.128906,0.128348,0.128348,469033600
1,1980-12-15,0.122210,0.122210,0.121652,0.121652,175884800
2,1980-12-16,0.113281,0.113281,0.112723,0.112723,105728000
3,1980-12-17,0.115513,0.116071,0.115513,0.115513,86441600
4,1980-12-18,0.118862,0.119420,0.118862,0.118862,73449600
...,...,...,...,...,...,...
10851,2023-12-28,194.139999,194.660004,193.169998,193.580002,34049900
10852,2023-12-29,193.899994,194.399994,191.729996,192.529999,42628800
10853,2024-01-02,187.149994,188.440002,183.889999,185.639999,82488700
10854,2024-01-03,184.220001,185.880005,183.429993,184.250000,58414500


In [48]:
actual = data.orderBy(desc('Date')).limit(30)
actual = actual.orderBy('Date', ascending=True)

In [49]:
actual.show(30)

+----------+------------------+------------------+------------------+------------------+---------+
|      Date|              Open|              High|               Low|             Close|   Volume|
+----------+------------------+------------------+------------------+------------------+---------+
|2023-11-21|191.41000366210938|191.52000427246094|189.74000549316406|190.63999938964844| 38134500|
|2023-11-22|191.49000549316406|192.92999267578125| 190.8300018310547|191.30999755859375| 39617700|
|2023-11-24| 190.8699951171875|190.89999389648438|            189.25|189.97000122070312| 24048300|
|2023-11-27| 189.9199981689453| 190.6699981689453|188.89999389648438| 189.7899932861328| 40552600|
|2023-11-28|189.77999877929688| 191.0800018310547|189.39999389648438|190.39999389648438| 38415400|
|2023-11-29|190.89999389648438|192.08999633789062|188.97000122070312| 189.3699951171875| 43014200|
|2023-11-30|189.83999633789062|190.32000732421875|188.19000244140625| 189.9499969482422| 48794400|
|2023-12-0

In [50]:
data = data.subtract(actual)
data = data.orderBy('Date', ascending=True)

In [51]:
data.toPandas()

,Date,Open,High,Low,Close,Volume
0,1980-12-12,0.128348,0.128906,0.128348,0.128348,469033600
1,1980-12-15,0.122210,0.122210,0.121652,0.121652,175884800
2,1980-12-16,0.113281,0.113281,0.112723,0.112723,105728000
3,1980-12-17,0.115513,0.116071,0.115513,0.115513,86441600
4,1980-12-18,0.118862,0.119420,0.118862,0.118862,73449600
...,...,...,...,...,...,...
10821,2023-11-14,187.699997,188.110001,186.300003,187.440002,60108400
10822,2023-11-15,187.850006,189.500000,187.779999,188.009995,53790500
10823,2023-11-16,189.570007,190.960007,188.649994,189.710007,54412900
10824,2023-11-17,190.250000,190.380005,188.570007,189.690002,50922700


In [52]:
#Tạo cột giá đóng sau 30 ngày tương ứng với từng ngày.
forecast_out =30
window_spec = Window().orderBy("Date")  
data_30 = data.withColumn("Close_after_30_days", F.lead("Close", forecast_out).over(window_spec))

In [53]:
data_30.show()

+----------+-------------------+-------------------+-------------------+-------------------+---------+-------------------+
|      Date|               Open|               High|                Low|              Close|   Volume|Close_after_30_days|
+----------+-------------------+-------------------+-------------------+-------------------+---------+-------------------+
|1980-12-12| 0.1283479928970337| 0.1289059966802597| 0.1283479928970337| 0.1283479928970337|469033600|0.14285700023174286|
|1980-12-15|0.12221000343561172|0.12221000343561172|0.12165199965238571|0.12165199965238571|175884800|0.13839299976825714|
|1980-12-16| 0.1132809966802597| 0.1132809966802597|0.11272300034761429|0.11272300034761429|105728000| 0.1333709955215454|
|1980-12-17|0.11551299691200256|0.11607100069522858|0.11551299691200256|0.11551299691200256| 86441600|0.12611599266529083|
|1980-12-18|0.11886200308799744|0.11941999942064285|0.11886200308799744|0.11886200308799744| 73449600|0.11886200308799744|
|1980-12-19|0.12

In [55]:
#Trong df mới thì 30 dòng cuối không có giá đóng sau 30 ngày nên dùng làm tập dự đoán (streaming)
df = data_30.toPandas()
df.tail(40)

,Date,Open,High,Low,Close,Volume,Close_after_30_days
10786,2023-09-26,174.820007,175.199997,171.660004,171.960007,64588900,181.820007
10787,2023-09-27,172.619995,173.039993,169.050003,170.429993,66921800,182.889999
10788,2023-09-28,169.339996,172.029999,167.619995,170.690002,56294400,182.410004
10789,2023-09-29,172.020004,173.070007,170.339996,171.210007,51814200,186.399994
10790,2023-10-02,171.220001,174.300003,170.929993,173.750000,52164500,184.800003
10791,2023-10-03,172.259995,173.630005,170.820007,172.399994,49594600,187.440002
10792,2023-10-04,171.089996,174.210007,170.970001,173.660004,53020300,188.009995
10793,2023-10-05,173.789993,175.449997,172.679993,174.910004,48527900,189.710007
10794,2023-10-06,173.800003,177.990005,173.179993,177.490005,57224100,189.690002
10795,2023-10-09,176.809998,179.050003,175.800003,178.990005,42390800,191.449997


In [56]:
stream = data_30.filter(isnull("Close_after_30_days"))
stream.show(30, truncate=False)

+----------+------------------+------------------+------------------+------------------+--------+-------------------+
|Date      |Open              |High              |Low               |Close             |Volume  |Close_after_30_days|
+----------+------------------+------------------+------------------+------------------+--------+-------------------+
|2023-10-10|178.10000610351562|179.72000122070312|177.9499969482422 |178.38999938964844|43698000|NULL               |
|2023-10-11|178.1999969482422 |179.85000610351562|177.60000610351562|179.8000030517578 |47551100|NULL               |
|2023-10-12|180.07000732421875|182.33999633789062|179.0399932861328 |180.7100067138672 |56743100|NULL               |
|2023-10-13|181.4199981689453 |181.92999267578125|178.13999938964844|178.85000610351562|51427100|NULL               |
|2023-10-16|176.75            |179.0800018310547 |176.50999450683594|178.72000122070312|52517000|NULL               |
|2023-10-17|176.64999389648438|178.4199981689453 |174.80

In [57]:
train = data_30.subtract(stream)

In [58]:
train.toPandas()

,Date,Open,High,Low,Close,Volume,Close_after_30_days
0,1980-12-12,0.128348,0.128906,0.128348,0.128348,469033600,0.142857
1,1980-12-15,0.122210,0.122210,0.121652,0.121652,175884800,0.138393
2,1980-12-16,0.113281,0.113281,0.112723,0.112723,105728000,0.133371
3,1980-12-17,0.115513,0.116071,0.115513,0.115513,86441600,0.126116
4,1980-12-18,0.118862,0.119420,0.118862,0.118862,73449600,0.118862
...,...,...,...,...,...,...,...
10791,2023-10-03,172.259995,173.630005,170.820007,172.399994,49594600,187.440002
10792,2023-10-04,171.089996,174.210007,170.970001,173.660004,53020300,188.009995
10793,2023-10-05,173.789993,175.449997,172.679993,174.910004,48527900,189.710007
10794,2023-10-06,173.800003,177.990005,173.179993,177.490005,57224100,189.690002


In [68]:
actual_pd = actual.toPandas()
actual_pd.to_csv("actual.csv", index=False)

In [69]:
stream_pd = stream.toPandas()
stream_pd.to_csv("stream.csv", index=False)

In [70]:
train_pd = train.toPandas()
train_pd.to_csv("train.csv", index=False)

Dự đoán